ATM: in: x out: 2

Wip1: in: x out: 2

Chk2: in: xy out: 1

Mdm2: in: x out: 1

p53: in: (x+y)z out: 2

the form for python is 

a x <= b
so the vector is (a,b)

the form for sage is 
b + ax >= 0

so the vector is (b,a)

In [3]:
from total2partial import *
import time 
import re
import json
import numpy as np
from itertools import product
from itertools import permutations
import os

# (1;1)

## 1. generate matrix for each atomic parameter node

In [44]:
# the old form for mathematica is Ax < b, x>0
def generatePMatrix(m, pmatrix):
    acc_prod = list(map(lambda x: int(x), [np.prod(m[i+1:]) for i in range(len(m))]))
    l = np.sum(m)-1
    p = [() for _ in range(np.prod(m))]
    level = []

    for i in range(len(p)):
        total = i
        level = []
        for j in range(len(m)):
            current = int(total/acc_prod[j])
            total %= acc_prod[j]
            level.append(current)
        curr_p = []
        for k in range(len(level)):
            temp = [0 for _ in range(m[k])]
            temp[m[k]-1-level[k]] = 1
            curr_p+=temp
        pmatrix.append(curr_p)

def rowByThresholdLinear(l,start,nothresholds,pmatrix):
    thresholds = [0 for _ in range(nothresholds)]
    currthreshold = -l[start]-1
    thresholds[currthreshold] = -1
    ret = []
    # lower part
    for i in range(start-1,-1,-1):
        if l[i] >=0:
            temp = list(pmatrix[l[i]])+thresholds+[0]
            ret.append(temp)
        else:
            break
    # upper part
    for i in range(start+1,len(l)):
        if l[i] >=0:
            temp = list(pmatrix[l[i]])+thresholds+[0]
            temp = [-x for x in temp]
            ret.append(temp)
        else:
            break
    return ret      
    
    
def generatePolynomialRowLinear(l,pmatrix,nothresholds):
    ret = []
    for i in range(len(l)):
        if l[i] <0:
            temp = rowByThresholdLinear(l,i,nothresholds,pmatrix)
            ret+=temp
    return ret

def generateULEpsilonLinear(noinedges,nothresholds,epsilon):
    ret = []
    
    for i in range(noinedges):
        
        # for u-l > epsilon
        temp = [0 for _ in range(noinedges*2+nothresholds+1)]
        temp[2*i] = -1
        temp[2*i+1] = 1
        temp[-1] = -epsilon
        ret.append(temp)
        
        # for l > epsilon
        temp = [0 for _ in range(noinedges*2+nothresholds+1)]
        temp[2*i+1] = -1
        temp[-1] = -epsilon
        ret.append(temp)
        
    return ret

def generateThresholdOrder(l):
    ret = []
    for i in range(len(l)):
        if l[i] <0:
            ret.append(-l[i])
    return ret

def generateNonNegtaiveLinear(noinedges,nothresholds,smallestThreshold,epsilon):
    # for l and theta, no need for u
    ret = []
    
    #for i in range(noinedges):
    #    # for l >= 0
    #    temp = [0 for _ in range(noinedges*2+nothresholds+1)]
    #    temp[2*i+1] = -1
    #    ret.append(temp)
    
    # non negative for the smallest threshold
    temp = [0 for _ in range(noinedges*2+nothresholds+1)]
    temp[noinedges*2+smallestThreshold-1]=-1
    temp[-1] = -epsilon*noinedges
    ret.append(temp)
    
    return ret

def generateThresholdOrder(l):
    ret = []
    for i in range(len(l)):
        if l[i] <0:
            ret.append(-l[i])
    return ret

def generateThresholdComparison(noinedges,nothresholds,thresholdorders):
    ret =  []
    # \sum  < level
    for i in range(1,len(thresholdorders)):
        temp = [0 for _ in range(noinedges*2+nothresholds+1)]
        biggerThreshold = thresholdorders[i]
        smallerThreshold = thresholdorders[i-1]
        temp[noinedges*2+biggerThreshold-1]=-1
        temp[noinedges*2+smallerThreshold-1] = 1
        ret.append(temp)
    
    return ret
    

def generateSDR(noinedges,nothresholds, level):
    ret =  []
    # \sum  < level
    temp = [1 for _ in range(noinedges*2+nothresholds+1)]
    temp[-1] = level
    ret.append(temp)
    
    # \sum > level
    temp = [-1 for _ in range(noinedges*2+nothresholds+1)]
    temp[-1] = -level
    ret.append(temp)
    
    return ret

def sagePTemplate(m):
    ret = []
    for i in range(len(m)):
        curr = m[i]
        first = [curr[-1]]
        second = [-x for x in curr[:-1]]
        temp = first + second
        ret.append(temp)
    return ret

# generate the matrix for parameter node
def generatePNMatrixLinear(ls,noinedges,nothresholds,epsilon, level,pmatrix):
    ret = []
    for i in range(len(ls)):
        l = ls[i]
        thresholdOrder = generateThresholdOrder(l)
        temp = generatePolynomialRowLinear(l,pmatrix, nothresholds)
        temp += generateULEpsilonLinear(noinedges,nothresholds,epsilon)
        temp += generateNonNegtaiveLinear(noinedges,nothresholds,thresholdOrder[0],epsilon)
        temp += generateSDR(noinedges,nothresholds,level)
        temp += generateThresholdComparison(noinedges,nothresholds,thresholdOrder)
        ret.append(sagePTemplate(temp))
        
    return ret

In [72]:
# x
start =time.time()
file = '2_final.dat'
print(file)
for i in range(1,2):
    rx1 = getPartialOrderByThreshold(i,file)
    print('Threshold: {} --> PG: {}'.format(i, len(rx1)))
time.time() -start

ls = list(rx1)
ilist = list(rx1)
noinedges = 1
nothresholds = 1
epsilon = 1/1000
pmatrix = []
pmatrix = []
generatePMatrix([2],pmatrix)
#thresholdsorder = (0,)
level = 1

2_final.dat
Threshold: 1 --> PG: 3


In [73]:
mats = generatePNMatrixLinear(ls,noinedges,nothresholds,epsilon,level,pmatrix)

In [74]:
mats[0]

[[0, 0, 1, -1],
 [0, 1, 0, -1],
 [-1/1000, 1, -1, 0],
 [-1/1000, 0, 1, 0],
 [-1/1000, 0, 0, 1],
 [1, -1, -1, -1],
 [-1, 1, 1, 1]]

In [75]:
# check the base regions is closed and bounded
basep = Polyhedron(ieqs= mats[0][2:])
basep.Vrepresentation()

(A vertex at (499/500, 1/1000, 1/1000),
 A vertex at (1/2, 499/1000, 1/1000),
 A vertex at (1/500, 1/1000, 997/1000))

In [76]:
basep = Polyhedron(ieqs= mats[0])
basep.Vrepresentation()

(A vertex at (499/500, 1/1000, 1/1000),
 A vertex at (1/2, 499/1000, 1/1000),
 A vertex at (167/500, 333/1000, 333/1000))

In [77]:
basep = Polyhedron(ieqs= mats[1])
basep.Vrepresentation()

(A vertex at (999/2000, 1/1000, 999/2000),
 A vertex at (1/500, 1/1000, 997/1000),
 A vertex at (1001/3000, 499/1500, 1001/3000))

In [78]:
basep = Polyhedron(ieqs= mats[2])
basep.Vrepresentation()

(A vertex at (499/500, 1/1000, 1/1000),
 A vertex at (999/2000, 1/1000, 999/2000),
 A vertex at (1001/3000, 499/1500, 1001/3000),
 A vertex at (167/500, 333/1000, 333/1000))

In [79]:
vlists = set()
plist = []
for mat in mats:
    p = Polyhedron(ieqs = mat)
    plist.append(p)
    vlists|= set(p.vertices())
vlists = list(vlists)
pdim = plist[0].dimension()

vdict = dict()
for i in range(len(vlists)):
    vdict[vlists[i]] = i

nodelist = dict()
# key is the index in ilist
# value is the dictionary dnode
# dnode takes dimension 0, ..., pdim as keys
# takes set/tuple(json) of tuples of vertices index (faces) as values
# the differential operator can be do by checking the subset of vertices index
for i in range(len(ilist)):
    key = ilist[i]
    value = dict()
    p = plist[i]
    for j in range(pdim+1):
        faces = p.faces(j)
        fvIndexSet = set()
        for f in faces:
            tempindex =[]
            for v in f.vertices():
                tempindex.append(vdict[v])
            fvIndexSet.add(tuple(sorted(tempindex)))
        value[j] = tuple(fvIndexSet)
    nodelist[key] = value
    

In [80]:
nodelist

{(-1, 0, 1): {0: ((2,), (1,), (4,)),
  1: ((1, 2), (2, 4), (1, 4)),
  2: ((1, 2, 4),)},
 (0, 1, -1): {0: ((5,), (0,), (3,)),
  1: ((0, 3), (0, 5), (3, 5)),
  2: ((0, 3, 5),)},
 (0, -1, 1): {0: ((2,), (0,), (3,), (4,)),
  1: ((0, 3), (2, 3), (2, 4), (0, 4)),
  2: ((0, 2, 3, 4),)}}

In [81]:
jsonNodelist = dict()
for key in nodelist:
    skey = str(key)
    jsonNodelist[skey] = nodelist[key]
    
with open('x_o_1.json','w') as file:
    json.dump(jsonNodelist,file)

In [55]:
firstKey = list(nodelist.keys())[0]
dim = len(nodelist[firstKey])-1
fList = []
for i in range(dim+1):
    dimFL = set()
    for cell in nodelist:
        dimFL |= set(nodelist[cell][i])
    fList.append(dimFL)

In [56]:
for i in range(dim+1):
    print(i, len(fList[i]))

0 6
1 8
2 3


# (2;1)

In [89]:
# x
start =time.time()
file = '4_final.dat'
print(file)
for i in range(1,2):
    rx1 = getPartialOrderByThreshold(i,file)
    print('Threshold: {} --> PG: {}'.format(i, len(rx1)))
time.time() -start

ls = list(rx1)
ilist = list(rx1)
noinedges = 2
nothresholds = 1
epsilon = 1/1000
pmatrix = []
pmatrix = []
generatePMatrix([2,2],pmatrix)
#thresholdsorder = (0,)
level = 1

4_final.dat
Threshold: 1 --> PG: 6


In [83]:
mats = generatePNMatrixLinear(ls,noinedges,nothresholds,epsilon,level,pmatrix)

In [84]:
mats[0]

[[0, 0, 1, 0, 1, -1],
 [0, 0, 1, 1, 0, -1],
 [0, 1, 0, 0, 1, -1],
 [0, 1, 0, 1, 0, -1],
 [-1/1000, 1, -1, 0, 0, 0],
 [-1/1000, 0, 1, 0, 0, 0],
 [-1/1000, 0, 0, 1, -1, 0],
 [-1/1000, 0, 0, 0, 1, 0],
 [-1/500, 0, 0, 0, 0, 1],
 [1, -1, -1, -1, -1, -1],
 [-1, 1, 1, 1, 1, 1]]

In [85]:
vlists = set()
plist = []
for mat in mats:
    p = Polyhedron(ieqs = mat)
    plist.append(p)
    vlists|= set(p.vertices())
vlists = list(vlists)
pdim = plist[0].dimension()

vdict = dict()
for i in range(len(vlists)):
    vdict[vlists[i]] = i

nodelist = dict()
# key is the index in ilist
# value is the dictionary dnode
# dnode takes dimension 0, ..., pdim as keys
# takes set/tuple(json) of tuples of vertices index (faces) as values
# the differential operator can be do by checking the subset of vertices index
for i in range(len(ilist)):
    key = ilist[i]
    value = dict()
    p = plist[i]
    for j in range(pdim+1):
        faces = p.faces(j)
        fvIndexSet = set()
        for f in faces:
            tempindex =[]
            for v in f.vertices():
                tempindex.append(vdict[v])
            fvIndexSet.add(tuple(sorted(tempindex)))
        value[j] = tuple(fvIndexSet)
    nodelist[key] = value
    

In [86]:
jsonNodelist = dict()
for key in nodelist:
    skey = str(key)
    jsonNodelist[skey] = nodelist[key]
    
with open('x+y_o_1.json','w') as file:
    json.dump(jsonNodelist,file)

In [87]:
firstKey = list(nodelist.keys())[0]
dim = len(nodelist[firstKey])-1
fList = []
for i in range(dim+1):
    dimFL = set()
    for cell in nodelist:
        dimFL |= set(nodelist[cell][i])
    fList.append(dimFL)

In [88]:
for i in range(dim+1):
    print(i, len(fList[i]))

0 18
1 48
2 54
3 29
4 6


# (3:1)

In [149]:
# x
start =time.time()
file = '8_final.dat'
print(file)
for i in range(1,2):
    rx1 = getPartialOrderByThreshold(i,file)
    print('Threshold: {} --> PG: {}'.format(i, len(rx1)))
time.time() -start

ls = list(rx1)
ilist = list(rx1)
noinedges = 3
nothresholds = 1
epsilon = 1/2**256
pmatrix = []
pmatrix = []
generatePMatrix([2,2,2],pmatrix)
#thresholdsorder = (0,)
level = 1

8_final.dat
Threshold: 1 --> PG: 20


In [150]:
mats = generatePNMatrixLinear(ls,noinedges,nothresholds,epsilon,level,pmatrix)

In [152]:
vlists = set()
plist = []
for mat in mats:
    p = Polyhedron(ieqs = mat)
    plist.append(p)
    vlists|= set(p.vertices())
vlists = list(vlists)
pdim = plist[0].dimension()

vdict = dict()
for i in range(len(vlists)):
    vdict[vlists[i]] = i

nodelist = dict()
# key is the index in ilist
# value is the dictionary dnode
# dnode takes dimension 0, ..., pdim as keys
# takes set/tuple(json) of tuples of vertices index (faces) as values
# the differential operator can be do by checking the subset of vertices index
for i in range(len(ilist)):
    key = ilist[i]
    value = dict()
    p = plist[i]
    for j in range(pdim+1):
        faces = p.faces(j)
        fvIndexSet = set()
        for f in faces:
            tempindex =[]
            for v in f.vertices():
                tempindex.append(vdict[v])
            fvIndexSet.add(tuple(sorted(tempindex)))
        value[j] = tuple(fvIndexSet)
    nodelist[key] = value
    

In [153]:
jsonNodelist = dict()
for key in nodelist:
    skey = str(key)
    jsonNodelist[skey] = nodelist[key]
    
with open('x+y+z_o_1.json','w') as file:
    json.dump(jsonNodelist,file)

In [154]:
firstKey = list(nodelist.keys())[0]
dim = len(nodelist[firstKey])-1
fList = []
for i in range(dim+1):
    dimFL = set()
    for cell in nodelist:
        dimFL |= set(nodelist[cell][i])
    fList.append(dimFL)

In [155]:
for i in range(dim+1):
    print(i, len(fList[i]))

0 57
1 243
2 484
3 551
4 366
5 132
6 20


# (4;1)

In [173]:
# x
start =time.time()
file = '16_final.dat'
print(file)
for i in range(1,2):
    rx1 = getPartialOrderByThreshold(i,file)
    print('Threshold: {} --> PG: {}'.format(i, len(rx1)))
time.time() -start

ls = list(rx1)
ilist = list(rx1)
noinedges = 4
nothresholds = 1
epsilon = 1/2**512
pmatrix = []
pmatrix = []
generatePMatrix([2,2,2,2],pmatrix)
#thresholdsorder = (0,)
level = 1

16_final.dat
Threshold: 1 --> PG: 150


In [174]:
mats = generatePNMatrixLinear(ls,noinedges,nothresholds,epsilon,level,pmatrix)

In [176]:
vlists = set()
plist = []
for mat in mats:
    p = Polyhedron(ieqs = mat)
    plist.append(p)
    vlists|= set(p.vertices())
vlists = list(vlists)
pdim = plist[0].dimension()

vdict = dict()
for i in range(len(vlists)):
    vdict[vlists[i]] = i

nodelist = dict()
# key is the index in ilist
# value is the dictionary dnode
# dnode takes dimension 0, ..., pdim as keys
# takes set/tuple(json) of tuples of vertices index (faces) as values
# the differential operator can be do by checking the subset of vertices index
for i in range(len(ilist)):
    print(i)
    key = ilist[i]
    value = dict()
    p = plist[i]
    for j in range(pdim+1):
        faces = p.faces(j)
        fvIndexSet = set()
        for f in faces:
            tempindex =[]
            for v in f.vertices():
                tempindex.append(vdict[v])
            fvIndexSet.add(tuple(sorted(tempindex)))
        value[j] = tuple(fvIndexSet)
    nodelist[key] = value
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149


In [177]:
jsonNodelist = dict()
for key in nodelist:
    skey = str(key)
    jsonNodelist[skey] = nodelist[key]
    
with open('x+y+z+w_o_1.json','w') as file:
    json.dump(jsonNodelist,file)

In [178]:
firstKey = list(nodelist.keys())[0]
dim = len(nodelist[firstKey])-1
fList = []
for i in range(dim+1):
    dimFL = set()
    for cell in nodelist:
        dimFL |= set(nodelist[cell][i])
    fList.append(dimFL)

In [179]:
for i in range(dim+1):
    print(i, len(fList[i]))

0 306
1 1902
2 5700
3 10380
4 12224
5 9370
6 4520
7 1247
8 150


# Case in: x out: 2 (ATM, Wip1)

If there are multiple thresholds we can not just compute the case $\theta_1 < \theta_2$ but both of them, as for regulatory network the $\theta_i$ are not in imsomorphic condition.

In [94]:
def thresholdSymmetry(l):
    l = list(l)
    indexList = []
    valuesList = []
    for i, value in enumerate(l):
        if value < 0:
            indexList.append(i)
            valuesList.append(l[i])
    
    ret = []
    for p in permutations(valuesList):
        temp = l.copy()
        for i in range(len(p)):
            temp[indexList[i]] = p[i]
        ret.append(tuple(temp))
    return ret

In [38]:
# prepare order list of atomic parameter node
start =time.time()
file = '2_final.dat'
print(file)
for i in range(2,3):
    rx2 = getPartialOrderByThreshold(i,file)
    print('Threshold: {} --> PG: {}'.format(i, len(rx2)))
time.time() -start

2_final.dat
Threshold: 2 --> PG: 6


0.001367807388305664

In [39]:
# compute matrix for each atomic parameter node
oldls = list(rx2)
noinedges = 1
nothresholds = 2
epsilon = 1/1000
pmatrix = []
generatePMatrix([2],pmatrix)
level = 1
thresholdsorder = (0,1)
ls = []

for i in oldls:
    ls+=thresholdSymmetry(i)

ilist = ls # the index list for atomic parameter node
mats = generatePNMatrixLinear(ls,noinedges,nothresholds,epsilon,level,pmatrix)

In [40]:
# check the base region is bounded closed 2 is the number of polynomials
basep = Polyhedron(ieqs = mats[0][2:])
basep.Vrepresentation()

(A vertex at (999/1000, 1/1000, 0, 0),
 A vertex at (1/500, 1/1000, 997/1000, 0),
 A vertex at (1001/2000, 999/2000, 0, 0),
 A vertex at (1/500, 1/1000, 997/2000, 997/2000))

In [98]:
vlists = set()
plist = []
for mat in mats:
    p = Polyhedron(ieqs = mat)
    plist.append(p)
    vlists|= set(p.vertices())
vlists = list(vlists)
pdim = plist[0].dimension()

vdict = dict()
for i in range(len(vlists)):
    vdict[vlists[i]] = i

nodelist = dict()
# key is the index in ilist
# value is the dictionary dnode
# dnode takes dimension 0, ..., pdim as keys
# takes set/tuple(json) of tuples of vertices index (faces) as values
# the differential operator can be do by checking the subset of vertices index
for i in range(len(ilist)):
    key = ilist[i]
    value = dict()
    p = plist[i]
    for j in range(pdim+1):
        faces = p.faces(j)
        fvIndexSet = set()
        for f in faces:
            tempindex =[]
            for v in f.vertices():
                tempindex.append(vdict[v])
            fvIndexSet.add(tuple(sorted(tempindex)))
        value[j] = tuple(fvIndexSet)
    nodelist[key] = value
    

In [99]:
jsonNodelist = dict()
for key in nodelist:
    skey = str(key)
    jsonNodelist[skey] = nodelist[key]
    
with open('xy_o_1.json','w') as file:
    json.dump(jsonNodelist,file)

In [100]:
firstKey = list(nodelist.keys())[0]
dim = len(nodelist[firstKey])-1
fList = []
for i in range(dim+1):
    dimFL = set()
    for cell in nodelist:
        dimFL |= set(nodelist[cell][i])
    fList.append(dimFL)

In [101]:
for i in range(dim+1):
    print(i, len(fList[i]))

0 18
1 48
2 54
3 29
4 6


# Nonlinear Case

# (1,1;1)

For product case, we should alwaya make restriction and then take log, as whenever we take the log 
the space will be unbounded. If we take log first, then it is not easy to find a bounded closed strong deformation retraction.

restriction over $\frac{u}{l} > 1+\epsilon$, $log(u) - log(l) > log(1+\epsilon)\approx \epsilon$, $l,\theta>\epsilon$ $\prod u, l,\theta =1$

In [190]:
def diffByOneBit(a,b):
    # a< b and differ only one bit
    diff = abs(a-b)
    if (a|b == b) and bin(diff).count('1') == 1:
        return True
    return False

def generateOneBitCondition(m):
    
    counts = [int(np.log2(i)) for i in m]
    bitsNo = [sum(counts[i+1:]) for i in range(len(counts))]
    
    pairs = set()
    for i in range(len(m)):
        component = m[i]
        for j in range(component):
            for k in range(j+1,component):
                if diffByOneBit(j,k):
                    pairs.add((j<<bitsNo[i],k<<bitsNo[i]))
            
    return pairs

def generateOneBitPolynomialLogLinear(m, p_matrix,nothresholds,epsilon):
    ret = []
    pairs = generateOneBitCondition(m)
    for l, u in pairs:
        smallerPoly = p_matrix[l]
        biggerPoly = p_matrix[u]
        diffPoly = [-biggerPoly[j]+smallerPoly[j] for j in range(len(biggerPoly))]
        diffPoly += [0 for j in range(nothresholds+1)]
        diffPoly[-1] = -epsilon
        ret.append(diffPoly)
    return ret


def generateThresholodLowerBoundLogLinear(m,nothresholds,smallestThreshold,lowerBound):
    ret = []
    
    temp = [0 for _ in range(sum(m)+nothresholds+1)]
    temp[sum(m)+smallestThreshold-1]=-1
    temp[-1] = -lowerBound
    ret.append(temp)
    
    return ret
    

def generateLsLowerBoundLogLinear(m,p_matrix,lowerBound,nothresholds):
    # for l_i sums
    ret = []
    smallestPoly = p_matrix[0]
    basis = 0
    rightList = [0 for _ in range(nothresholds+1)]
    rightList[-1] = -lowerBound
    #print(smallestPoly)
    for i in range(len(m)):
        #print(smallestPoly[basis:basis+m[i]],basis,m[i])
        temp = [0 for _ in range(basis)]+smallestPoly[basis:basis+m[i]]+[0 for _ in range(basis+m[i],len(smallestPoly))]
        temp = [-x for x in temp]+rightList
        ret.append(temp)
        basis +=m[i]
    
    return ret


def generateThresholdComparisonLogLinear(m,nothresholds,thresholdorders):
    ret =  []
    # \sum  < level
    base = sum(m)
    for i in range(1,len(thresholdorders)):
        temp = [0 for _ in range(base+nothresholds+1)]
        biggerThreshold = thresholdorders[i]
        smallerThreshold = thresholdorders[i-1]
        temp[base+biggerThreshold-1]=-1
        temp[base+smallerThreshold-1] = 1
        ret.append(temp)
    
    return ret
    


def generateSDRLogLinear(m,nothresholds,level):
    ret =  []
    base = sum(m)
    # \sum  <= level
    temp = [1 for _ in range(base+nothresholds+1)]
    temp[-1] = level
    ret.append(temp)
    
    # \sum >= level
    temp = [-1 for _ in range(base+nothresholds+1)]
    temp[-1] = -level
    ret.append(temp)
    
    # also need the parameter paper restrction -x_0 + x_1+x_2 - x_3 >=0
    
    return ret

def generatePaperRestriction(m,nothresholds,epsilon):
    base = sum(m)
    temp = [0 for _ in range(base+nothresholds+1)]
    temp[0:4] = [1,-1,-1,1]
    temp[-1] = -epsilon
    return [temp]


# generate the matrix for parameter node
def generatePNMatrixLogLinear(ls,m,nothresholds,epsilon,lowerbound,thetalowerbound,level,pmatrix):
    ret = []
    for i in range(len(ls)):
        l = ls[i]
        thresholdOrder = generateThresholdOrder(l)
        temp = generatePolynomialRowLinear(l,pmatrix, nothresholds)
        temp += generateOneBitPolynomialLogLinear(m,pmatrix,nothresholds,epsilon)
        temp += generateLsLowerBoundLogLinear(m,pmatrix,lowerbound,nothresholds)
        temp += generateThresholodLowerBoundLogLinear(m,nothresholds,thresholdOrder[0],thetalowerbound)
        temp += generateSDRLogLinear(m,nothresholds,level)
        temp += generateThresholdComparisonLogLinear(m,nothresholds,thresholdOrder)
        ret.append(sagePTemplate(temp))
        
    return ret

# for the generator of (x+y)z
def generatePNMatrixLogLinearNotPure(ls,m,nothresholds,epsilon,lowerbound,thetalowerbound,level,pmatrix):
    ret = []
    for i in range(len(ls)):
        l = ls[i]
        thresholdOrder = generateThresholdOrder(l)
        temp = generatePolynomialRowLinear(l,pmatrix, nothresholds)
        temp += generateOneBitPolynomialLogLinear(m,pmatrix,nothresholds,epsilon)
        temp += generateLsLowerBoundLogLinear(m,pmatrix,lowerbound,nothresholds)
        temp += generateThresholodLowerBoundLogLinear(m,nothresholds,thresholdOrder[0],thetalowerbound)
        temp += generateSDRLogLinear(m,nothresholds,level)
        temp += generateThresholdComparisonLogLinear(m,nothresholds,thresholdOrder)
        temp += generatePaperRestriction(m,nothresholds,epsilon)
        ret.append(sagePTemplate(temp))
        
    return ret

temp

In [114]:
# prepare order list of atomic parameter node
start =time.time()
file = '2_2_final.dat'
print(file)
for i in range(1,2):
    rxy1 = getPartialOrderByThreshold(i,file)
    print('Threshold: {} --> PG: {}'.format(i, len(rxy1)))
time.time() -start

2_2_final.dat
Threshold: 1 --> PG: 6


0.002048015594482422

In [115]:
# compute matrix for each atomic parameter node
oldls = list(rxy1)
m = [2,2]
nothresholds = 1
epsilon = 1/1000
lowerbound = -10
# thetalowerbound = lowerbound*inedges
thetalowerbound = 2*lowerbound
pmatrix = []
generatePMatrix([2,2],pmatrix)
level = 0
ls = []

for i in oldls:
    ls+=thresholdSymmetry(i)

ilist = ls # the index list for atomic parameter node
mats = generatePNMatrixLogLinear(ls,m,nothresholds,epsilon,lowerbound,thetalowerbound,level,pmatrix)

In [116]:
mats[0]

[[0, 0, 1, 0, 1, -1],
 [0, 0, 1, 1, 0, -1],
 [0, 1, 0, 0, 1, -1],
 [0, 1, 0, 1, 0, -1],
 [-1/1000, 0, 0, 1, -1, 0],
 [-1/1000, 1, -1, 0, 0, 0],
 [10, 0, 1, 0, 0, 0],
 [10, 0, 0, 0, 1, 0],
 [20, 0, 0, 0, 0, 1],
 [0, -1, -1, -1, -1, -1],
 [0, 1, 1, 1, 1, 1]]

In [117]:
# check whether the base regions is bounded closed
basep = Polyhedron(ieqs = mats[0][4:])
basep.Vrepresentation()

(A vertex at (49999/1000, -10, -9999/1000, -10, -20),
 A vertex at (20, 19999/1000, -9999/1000, -10, -20),
 A vertex at (-9999/1000, -10, 49999/1000, -10, -20),
 A vertex at (-9999/1000, -10, -9999/1000, -10, 19999/500),
 A vertex at (-9999/1000, -10, 20, 19999/1000, -20))

In [118]:
vlists = set()
plist = []
for mat in mats:
    p = Polyhedron(ieqs = mat)
    plist.append(p)
    vlists|= set(p.vertices())
vlists = list(vlists)
pdim = plist[0].dimension()

vdict = dict()
for i in range(len(vlists)):
    vdict[vlists[i]] = i

nodelist = dict()
# key is the index in ilist
# value is the dictionary dnode
# dnode takes dimension 0, ..., pdim as keys
# takes set/tuple(json) of tuples of vertices index (faces) as values
# the differential operator can be do by checking the subset of vertices index
for i in range(len(ilist)):
    key = ilist[i]
    value = dict()
    p = plist[i]
    for j in range(pdim+1):
        faces = p.faces(j)
        fvIndexSet = set()
        for f in faces:
            tempindex =[]
            for v in f.vertices():
                tempindex.append(vdict[v])
            fvIndexSet.add(tuple(sorted(tempindex)))
        value[j] = tuple(fvIndexSet)
    nodelist[key] = value

In [119]:
jsonNodelist = dict()
for key in nodelist:
    skey = str(key)
    jsonNodelist[skey] = nodelist[key]
    
with open('xy_o_1.json','w') as file:
    json.dump(jsonNodelist,file)

In [120]:
firstKey = list(nodelist.keys())[0]
dim = len(nodelist[firstKey])-1
fList = []
for i in range(dim+1):
    dimFL = set()
    for cell in nodelist:
        dimFL |= set(nodelist[cell][i])
    fList.append(dimFL)

In [121]:
for i in range(dim+1):
    print(i, len(fList[i]))

0 18
1 48
2 54
3 29
4 6


# (1,1,1:1)

In [157]:
# prepare order list of atomic parameter node
start =time.time()
file = '2_2_2_final.dat'
print(file)
for i in range(1,2):
    rxy1 = getPartialOrderByThreshold(i,file)
    print('Threshold: {} --> PG: {}'.format(i, len(rxy1)))
time.time() -start

2_2_2_final.dat
Threshold: 1 --> PG: 20


0.003187894821166992

In [159]:
# compute matrix for each atomic parameter node
oldls = list(rxy1)
m = [2,2,2]
nothresholds = 1
epsilon = 1/2**256
lowerbound = -10
# thetalowerbound = lowerbound*inedges
thetalowerbound = 3*lowerbound
pmatrix = []
generatePMatrix([2,2,2],pmatrix)
level = 0
ls = []

for i in oldls:
    ls+=thresholdSymmetry(i)

ilist = ls # the index list for atomic parameter node
mats = generatePNMatrixLogLinear(ls,m,nothresholds,epsilon,lowerbound,thetalowerbound,level,pmatrix)

In [161]:
vlists = set()
plist = []
for mat in mats:
    p = Polyhedron(ieqs = mat)
    plist.append(p)
    vlists|= set(p.vertices())
vlists = list(vlists)
pdim = plist[0].dimension()

vdict = dict()
for i in range(len(vlists)):
    vdict[vlists[i]] = i

nodelist = dict()
# key is the index in ilist
# value is the dictionary dnode
# dnode takes dimension 0, ..., pdim as keys
# takes set/tuple(json) of tuples of vertices index (faces) as values
# the differential operator can be do by checking the subset of vertices index
for i in range(len(ilist)):
    key = ilist[i]
    value = dict()
    p = plist[i]
    for j in range(pdim+1):
        faces = p.faces(j)
        fvIndexSet = set()
        for f in faces:
            tempindex =[]
            for v in f.vertices():
                tempindex.append(vdict[v])
            fvIndexSet.add(tuple(sorted(tempindex)))
        value[j] = tuple(fvIndexSet)
    nodelist[key] = value

In [162]:
jsonNodelist = dict()
for key in nodelist:
    skey = str(key)
    jsonNodelist[skey] = nodelist[key]
    
with open('xyz_o_1.json','w') as file:
    json.dump(jsonNodelist,file)

In [163]:
firstKey = list(nodelist.keys())[0]
dim = len(nodelist[firstKey])-1
fList = []
for i in range(dim+1):
    dimFL = set()
    for cell in nodelist:
        dimFL |= set(nodelist[cell][i])
    fList.append(dimFL)

In [164]:
for i in range(dim+1):
    print(i, len(fList[i]))

0 57
1 243
2 484
3 551
4 366
5 132
6 20


# (1,1,1,1:1)

In [166]:
# prepare order list of atomic parameter node
start =time.time()
file = '2_2_2_2_final.dat'
print(file)
for i in range(1,2):
    rxy1 = getPartialOrderByThreshold(i,file)
    print('Threshold: {} --> PG: {}'.format(i, len(rxy1)))
time.time() -start

# compute matrix for each atomic parameter node
oldls = list(rxy1)
m = [2,2,2,2]
nothresholds = 1
epsilon = 1/2**512
lowerbound = -10
# thetalowerbound = lowerbound*inedges
thetalowerbound = 4*lowerbound
pmatrix = []
generatePMatrix([2,2,2,2],pmatrix)
level = 0
ls = []

for i in oldls:
    ls+=thresholdSymmetry(i)

ilist = ls # the index list for atomic parameter node
mats = generatePNMatrixLogLinear(ls,m,nothresholds,epsilon,lowerbound,thetalowerbound,level,pmatrix)

2_2_2_2_final.dat
Threshold: 1 --> PG: 150


In [169]:
vlists = set()
plist = []
count = 0
for mat in mats:
    p = Polyhedron(ieqs = mat)
    plist.append(p)
    vlists|= set(p.vertices())
vlists = list(vlists)
pdim = plist[0].dimension()

print("done 1")

vdict = dict()
for i in range(len(vlists)):
    vdict[vlists[i]] = i

print("done 2")

nodelist = dict()
# key is the index in ilist
# value is the dictionary dnode
# dnode takes dimension 0, ..., pdim as keys
# takes set/tuple(json) of tuples of vertices index (faces) as values
# the differential operator can be do by checking the subset of vertices index
for i in range(len(ilist)):
    print(i)
    key = ilist[i]
    value = dict()
    p = plist[i]
    for j in range(pdim+1):
        faces = p.faces(j)
        fvIndexSet = set()
        for f in faces:
            tempindex =[]
            for v in f.vertices():
                tempindex.append(vdict[v])
            fvIndexSet.add(tuple(sorted(tempindex)))
        value[j] = tuple(fvIndexSet)
    nodelist[key] = value

done 1
done 2
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149


In [170]:
jsonNodelist = dict()
for key in nodelist:
    skey = str(key)
    jsonNodelist[skey] = nodelist[key]
    
with open('xyzw_o_1.json','w') as file:
    json.dump(jsonNodelist,file)

In [171]:
firstKey = list(nodelist.keys())[0]
dim = len(nodelist[firstKey])-1
fList = []
for i in range(dim+1):
    dimFL = set()
    for cell in nodelist:
        dimFL |= set(nodelist[cell][i])
    fList.append(dimFL)

In [172]:
for i in range(dim+1):
    print(i, len(fList[i]))

0 306
1 1902
2 5700
3 10380
4 12224
5 9370
6 4520
7 1247
8 150


# Case in: (x+y)z out: 2 (p53)

For this case, we first explain the correspondence bewtween the unbouned cells of (x+y)z and linearized case, then make restriction $\epsilon$ in original parameters but transform them to 
the linearized space for computation problem. The regularity can be proved by using the strong 
deformation retraction technique for parameter logic $(2,1,\dots,1)$ rather than direct computation.

The base region also include onebit condition componentwise.

In [107]:
start =time.time()
file = '4_2_final.dat'
print(file)
for i in range(2,3):
    rxyz2 = getPartialOrderByThreshold(i,file)
    print('Threshold: {} --> PG: {}'.format(i, len(rxyz2)))
time.time() -start

4_2_final.dat
Threshold: 2 --> PG: 155


0.008254051208496094

In [108]:
# compute matrix for each atomic parameter node
oldls = list(rxyz2)
m = [4,2]
noinedges = 3
nothresholds = 2
epsilon = 1/1000
lowerbound = -10
thetalowerbound = 2*lowerbound # homogeneity degree
pmatrix = []

generatePMatrix([4,2],pmatrix)
level = 0
thresholdsorder = (0,1)
ls = []

for i in oldls:
    ls+=thresholdSymmetry(i)


ilist = ls # the index list for atomic parameter node
mats = generatePNMatrixLogLinear(ls,m,nothresholds,epsilon,lowerbound,thetalowerbound,level,pmatrix)

In [111]:
mats[0][-11:]

[[-1/1000, 0, 0, 0, 0, 1, -1, 0, 0],
 [-1/1000, 1, 0, -1, 0, 0, 0, 0, 0],
 [-1/1000, 1, -1, 0, 0, 0, 0, 0, 0],
 [-1/1000, 0, 1, 0, -1, 0, 0, 0, 0],
 [-1/1000, 0, 0, 1, -1, 0, 0, 0, 0],
 [10, 0, 0, 0, 1, 0, 0, 0, 0],
 [10, 0, 0, 0, 0, 0, 1, 0, 0],
 [20, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, -1, -1, -1, -1, -1, -1, -1, -1],
 [0, 1, 1, 1, 1, 1, 1, 1, 1],
 [0, 0, 0, 0, 0, 0, 0, 1, -1]]

In [112]:
# check the base region is bounded and closed
# look it is not closed as the onebit is not complete and we should also add -X_0+X_1+X_2-X_3 >=0
basep = Polyhedron(ieqs =mats[0][-11:])
basep.Vrepresentation()

(A vertex at (60003/4000, 59999/4000, 59999/4000, 11999/800, -9999/1000, -10, -20, -20),
 A vertex at (-4999/500, -9999/1000, -9999/1000, -10, -9999/1000, -10, 11999/400, 11999/400),
 A vertex at (-4999/500, -9999/1000, -9999/1000, -10, -9999/1000, -10, 15999/200, -20),
 A vertex at (-4999/500, -9999/1000, -9999/1000, -10, 79997/2000, 15999/400, -20, -20),
 A vertex at (-4999/500, -9999/1000, -9999/1000, -10, 22499/250, -10, -20, -20),
 A vertex at (79999/2000, 79997/2000, -9999/1000, -10, -9999/1000, -10, -20, -20),
 A vertex at (89997/1000, -9999/1000, -9999/1000, -10, -9999/1000, -10, -20, -20),
 A vertex at (79999/2000, -9999/1000, 79997/2000, -10, -9999/1000, -10, -20, -20),
 A vertex at (70001/3000, 34999/1500, 34999/1500, -10, -9999/1000, -10, -20, -20))

In [113]:
# generate set of vertices
vlists = set()
plist = []
for mat in mats:
    p = Polyhedron(ieqs = mat)
    plist.append(p)
    vlists|= set(p.vertices())
vlists = list(vlists)
pdim = plist[0].dimension()

vdict = dict()
for i in range(len(vlists)):
    vdict[vlists[i]] = i

In [114]:
nodelist = dict()
for i in range(len(ilist)):
    print(i)
    key = ilist[i]
    value = dict()
    p = plist[i]
    for j in range(pdim+1):
        faces = p.faces(j)
        fvIndexSet = set()
        for f in faces:
            tempindex =[]
            for v in f.vertices():
                tempindex.append(vdict[v])
            fvIndexSet.add(tuple(sorted(tempindex)))
        value[j] = tuple(fvIndexSet)
    nodelist[key] = value

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [116]:
# check the length of 0 and 1 cellss
s0 = set()
s1 = set()
for key in nodelist:
    cell0 = nodelist[key][0]
    cell1 = nodelist[key][1]
    
    for i in cell0:
        s0.add(len(i))
    for i in cell1:
        s1.add(len(i))

In [118]:
s0,s1

({1}, {2})

In [119]:
# modify the key to string and store to json files 
jsonNodelist = dict()
for key in nodelist:
    skey = str(key)
    jsonNodelist[skey] = nodelist[key]

with open('(x+y)z_o_2.json','w') as file:
    json.dump(jsonNodelist,file)

In [120]:
pwd

'/Users/lunzhang/Desktop/datapool/Homology'

In [144]:
def generate_p_matrix(m, p_matrix):
    acc_prod = list(map(lambda x: int(x), [np.prod(m[i+1:]) for i in range(len(m))]))
    l = np.sum(m)-1
    p = [() for _ in range(np.prod(m))]
    level = []

    for i in range(len(p)):
        total = i 
        level = []
        for j in range(len(m)):
            current = int(total/acc_prod[j])
            total %= acc_prod[j]
            level.append(current)
        curr_p = []
        for i in range(len(level)):
            temp = [0 for _ in range(m[i])]
            temp[m[i]-1-level[i]] = 1 
            curr_p+=temp
        p_matrix.append(curr_p)
        #print(p_matrix)

# (2,1;1)

In [181]:
start =time.time()
file = '4_2_final.dat'
print(file)
for i in range(1,2):
    rxyz2 = getPartialOrderByThreshold(i,file)
    print('Threshold: {} --> PG: {}'.format(i, len(rxyz2)))
time.time() -start

4_2_final.dat
Threshold: 1 --> PG: 20


0.002707958221435547

In [192]:
# compute matrix for each atomic parameter node
oldls = list(rxyz2)
m = [4,2]
noinedges = 3
nothresholds = 1
epsilon = 1/2**256
lowerbound = -10
thetalowerbound = 2*lowerbound # homogeneity degree
pmatrix = []

generatePMatrix([4,2],pmatrix)
level = 0
thresholdsorder = (0,)
ls = []

for i in oldls:
    ls+=thresholdSymmetry(i)


ilist = ls # the index list for atomic parameter node
mats = generatePNMatrixLogLinearNotPure(ls,m,nothresholds,epsilon,lowerbound,thetalowerbound,level,pmatrix)

In [193]:
vlists = set()
plist = []
count = 0
for mat in mats:
    p = Polyhedron(ieqs = mat)
    plist.append(p)
    vlists|= set(p.vertices())
vlists = list(vlists)
pdim = plist[0].dimension()

print("done 1")

vdict = dict()
for i in range(len(vlists)):
    vdict[vlists[i]] = i

print("done 2")

nodelist = dict()
# key is the index in ilist
# value is the dictionary dnode
# dnode takes dimension 0, ..., pdim as keys
# takes set/tuple(json) of tuples of vertices index (faces) as values
# the differential operator can be do by checking the subset of vertices index
for i in range(len(ilist)):
    print(i)
    key = ilist[i]
    value = dict()
    p = plist[i]
    for j in range(pdim+1):
        faces = p.faces(j)
        fvIndexSet = set()
        for f in faces:
            tempindex =[]
            for v in f.vertices():
                tempindex.append(vdict[v])
            fvIndexSet.add(tuple(sorted(tempindex)))
        value[j] = tuple(fvIndexSet)
    nodelist[key] = value

done 1
done 2
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [194]:
mats[0]

[[0, -1, 0, 0, 0, -1, 0, 1],
 [0, -1, 0, 0, 0, 0, -1, 1],
 [0, 0, -1, 0, 0, -1, 0, 1],
 [0, 0, -1, 0, 0, 0, -1, 1],
 [0, 0, 0, -1, 0, -1, 0, 1],
 [0, 0, 0, -1, 0, 0, -1, 1],
 [0, 0, 0, 0, -1, -1, 0, 1],
 [0, 0, 0, 0, -1, 0, -1, 1],
 [-1/115792089237316195423570985008687907853269984665640564039457584007913129639936,
  0,
  0,
  0,
  0,
  1,
  -1,
  0],
 [-1/115792089237316195423570985008687907853269984665640564039457584007913129639936,
  1,
  0,
  -1,
  0,
  0,
  0,
  0],
 [-1/115792089237316195423570985008687907853269984665640564039457584007913129639936,
  1,
  -1,
  0,
  0,
  0,
  0,
  0],
 [-1/115792089237316195423570985008687907853269984665640564039457584007913129639936,
  0,
  1,
  0,
  -1,
  0,
  0,
  0],
 [-1/115792089237316195423570985008687907853269984665640564039457584007913129639936,
  0,
  0,
  1,
  -1,
  0,
  0,
  0],
 [10, 0, 0, 0, 1, 0, 0, 0],
 [10, 0, 0, 0, 0, 0, 1, 0],
 [20, 0, 0, 0, 0, 0, 0, 1],
 [0, -1, -1, -1, -1, -1, -1, -1],
 [0, 1, 1, 1, 1, 1, 1, 1],
 [-1/11579208

In [195]:
jsonNodelist = dict()
for key in nodelist:
    skey = str(key)
    jsonNodelist[skey] = nodelist[key]
    
with open('(x+y)z_o_1.json','w') as file:
    json.dump(jsonNodelist,file)

In [196]:
firstKey = list(nodelist.keys())[0]
dim = len(nodelist[firstKey])-1
fList = []
for i in range(dim+1):
    dimFL = set()
    for cell in nodelist:
        dimFL |= set(nodelist[cell][i])
    fList.append(dimFL)

In [197]:
for i in range(dim+1):
    print(i, len(fList[i]))

0 66
1 272
2 524
3 581
4 378
5 134
6 20


# (2,1,1:1)

In [198]:
start =time.time()
file = '4_2_2_final.dat'
print(file)
for i in range(1,2):
    rxyz2 = getPartialOrderByThreshold(i,file)
    print('Threshold: {} --> PG: {}'.format(i, len(rxyz2)))
time.time() -start

4_2_2_final.dat
Threshold: 1 --> PG: 155


0.1578991413116455

In [201]:
# compute matrix for each atomic parameter node
oldls = list(rxyz2)
m = [4,2,2]
noinedges = 4
nothresholds = 1
epsilon = 1/2**512
lowerbound = -10
thetalowerbound = 3*lowerbound # homogeneity degree
pmatrix = []

generatePMatrix([4,2,2],pmatrix)
level = 0
thresholdsorder = (0,)
ls = []

for i in oldls:
    ls+=thresholdSymmetry(i)


ilist = ls # the index list for atomic parameter node
mats = generatePNMatrixLogLinearNotPure(ls,m,nothresholds,epsilon,lowerbound,thetalowerbound,level,pmatrix)

In [202]:
vlists = set()
plist = []
count = 0
for mat in mats:
    p = Polyhedron(ieqs = mat)
    plist.append(p)
    vlists|= set(p.vertices())
vlists = list(vlists)
pdim = plist[0].dimension()

print("done 1")

vdict = dict()
for i in range(len(vlists)):
    vdict[vlists[i]] = i

print("done 2")

nodelist = dict()
# key is the index in ilist
# value is the dictionary dnode
# dnode takes dimension 0, ..., pdim as keys
# takes set/tuple(json) of tuples of vertices index (faces) as values
# the differential operator can be do by checking the subset of vertices index
for i in range(len(ilist)):
    print(i)
    key = ilist[i]
    value = dict()
    p = plist[i]
    for j in range(pdim+1):
        faces = p.faces(j)
        fvIndexSet = set()
        for f in faces:
            tempindex =[]
            for v in f.vertices():
                tempindex.append(vdict[v])
            fvIndexSet.add(tuple(sorted(tempindex)))
        value[j] = tuple(fvIndexSet)
    nodelist[key] = value

done 1
done 2
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154


In [203]:
jsonNodelist = dict()
for key in nodelist:
    skey = str(key)
    jsonNodelist[skey] = nodelist[key]
    
with open('(x+y)zw_o_1.json','w') as file:
    json.dump(jsonNodelist,file)

In [204]:
firstKey = list(nodelist.keys())[0]
dim = len(nodelist[firstKey])-1
fList = []
for i in range(dim+1):
    dimFL = set()
    for cell in nodelist:
        dimFL |= set(nodelist[cell][i])
    fList.append(dimFL)

In [205]:
for i in range(dim+1):
    print(i, len(fList[i]))

0 433
1 2593
2 7406
3 12842
4 14451
5 10634
6 4948
7 1323
8 155


# (2,1,1,1;1)

In [206]:
start =time.time()
file = '4_2_2_2_final.dat'
print(file)
for i in range(1,2):
    rxyz2 = getPartialOrderByThreshold(i,file)
    print('Threshold: {} --> PG: {}'.format(i, len(rxyz2)))
time.time() -start

4_2_2_2_final.dat


FileNotFoundError: [Errno 2] No such file or directory: '4_2_2_2_final.dat'